In [1]:
import time
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from selenium.common. exceptions import TimeoutException
from bs4 import BeautifulSoup
from selenium.webdriver.chrome.options import Options
import pandas as pd

접속하기 

In [2]:
def 서버통신에러():
    try:
        WebDriverWait(driver,3).until(EC.alert_is_present())
        driver.switch_to_alert().accept()
    except:
        pass
    
def 접속하기(아이디, 비밀번호):
    for i in range(1,5):
        driver.get('https://www.open.go.kr/pa/main/paMain.do')
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'on')))
        try:
            driver.find_element_by_link_text('로그인').click()
            WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.CLASS_NAME, 'mberIdIcon')))
            driver.find_element_by_id('mberId').send_keys(아이디)
            driver.find_element_by_id('pwd').send_keys(비밀번호)
            driver.find_element_by_id('loginSubmitBtn').click()
        except:
            pass
        WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.CLASS_NAME, 'on')))
        if str(driver.page_source).count('로그아웃') >0:
            break
    driver.find_element_by_link_text("마이페이지").click()
    서버통신에러()
    WebDriverWait(driver,10).until(EC.element_to_be_clickable((By.PARTIAL_LINK_TEXT, '더보기')))
    element = driver.find_element_by_xpath('//*[@id="content"]/div[2]/div[3]/p/span/a')
    driver.execute_script("arguments[0].click();", element)

데이터 저장

In [3]:
def 저장하기(대학교_정보들):
    대학이름 = 대학교_정보들['대학이름']
    try:
        저장된_정보들 = 저장소.loc[대학이름].to_dict()
    except:
        #기존 저장소에 새로운 대학교 추가하기
        저장소.loc[대학이름] = pd.Series(대학교_정보들)
        
    else:
        #기존 대학교 정보를 업데이트
        저장된_정보들.update(대학교_정보들)
        저장소.loc[대학이름] = 저장된_정보들

공통함수

In [11]:
def 크롬_옵션설정_확인():
    options = Options()
    prefs = {'profile.default_content_setting_values.automatic_downloads': 1}
    options.add_experimental_option("prefs", prefs)
    return options
    
def 목록갯수_확인():
    return driver.find_element_by_xpath('//*[@id="totalCount"]').text

def 목록버튼_누르기():
    WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH,'//*[@id="listBnt"]')))
    목록버튼 = driver.find_element_by_xpath('//*[@id="listBnt"]')
    driver.execute_script("arguments[0].click();", 목록버튼)
    
def 원하는_페이지_누르기(페이지번호):
    element = driver.find_element_by_xpath('//*[@id="content"]/div[2]/div[3]/a['+str(페이지번호+1)+']')
    driver.execute_script("arguments[0].click();", element)
    서버통신에러()


def 조건_누르기(청구일자="최근3개월", 처리상태 ="전체"):
    #청구일자 선택
    WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.PARTIAL_LINK_TEXT, 청구일자)))
    element = driver.find_element_by_partial_link_text(청구일자)
    driver.execute_script("arguments[0].click();", element)
    

    #처리상태 선택
    select = Select(driver.find_element_by_name('prcsStsCdSch'))
    select.select_by_visible_text(처리상태)


    #검색버튼 누르기
    element = driver.find_element_by_xpath('//*[@id="searchBtn"]')
    driver.execute_script("arguments[0].click();", element)
    서버통신에러()

def 청구신청_하나_누르기(count):
    WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="openBillingTable"]/tbody/tr['+str(count)+']/td[4]/a')))
    element =driver.find_element_by_xpath('//*[@id="openBillingTable"]/tbody/tr['+str(count)+']/td[4]/a')
    driver.execute_script("arguments[0].click();", element)

def 청구신청조회_저장():
    soup = BeautifulSoup(driver.page_source,'html.parser')
    tbody = soup.find_all('tbody')[1]
    tr_s = tbody.find_all('tr')
    임시_딕셔너리 = dict()

    for tr in tr_s:
        td_s = tr.find_all('td')
        임시_딕셔너리['대학이름'] = td_s[4].text
        임시_딕셔너리['결정'] = td_s[5].text
        임시_딕셔너리['통지(처리)날짜'] = td_s[6].text
        저장하기(임시_딕셔너리)

def 다음페이지_누르기(넘기는횟수=1):
    while 넘기는횟수:
        WebDriverWait(driver,5).until(EC.element_to_be_clickable((By.CLASS_NAME,'direction ')))
        페이지버튼 = driver.find_elements_by_class_name('direction ')[2]
        driver.execute_script("arguments[0].click();",페이지버튼)
        if 넘기는횟수 ==1:
            return 0
        else:
            time.sleep(0.3)
            넘기는횟수 -=1
            

def 현재페이지_확인():
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    페이지들 = soup.find('div',class_='pagination mb20')
    현재페이지_ =페이지들.find('strong').text
    return 현재페이지_

공개, 비공개 처리

In [5]:
def 결정통지_파일다운로드(갯수):
    if 갯수 ==1:
        파일다운로드 = driver.find_element_by_xpath('//*[@id="dntcFileListTxt"]/a')
        driver.execute_script("arguments[0].click();", 파일다운로드)
    elif 갯수 >= 2:
        for i in range(1,갯수+1):
            파일다운로드 = driver.find_element_by_xpath('//*[@id="dntcFileListTxt"]/a['+str(i)+']')
            driver.execute_script("arguments[0].click();", 파일다운로드)

def 결정통지_누르기():
    WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH,'//*[@id="listBtn2"]/a')))
    결정통지버튼 = driver.find_element_by_xpath('//*[@id="resultBtn2"]/a')
    driver.execute_script("arguments[0].click();", 결정통지버튼)

def 결정통지_저장(다운로드_안하기):
    soup= BeautifulSoup(driver.page_source, "html.parser")
    파일갯수 = 파일갯수_확인()
    임시_딕셔너리 =dict()
    임시_딕셔너리['대학이름']=soup.find('td',id='prcsNstNm').text
    임시_딕셔너리['공개사유']=soup.find('td',id='oppCn').text.replace('\n','')
    임시_딕셔너리['비공개사유']=soup.find('td',id='dntcClsdrResnCn').text.replace('\n','')
    임시_딕셔너리['데이터유무'] =파일갯수
    if '안' in 다운로드_안하기:
        pass
    else:
        결정통지_파일다운로드(파일갯수)
    저장하기(임시_딕셔너리)

def 본인확인_누르기():
    try:
        WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.ID, 'modalIfm')))
        driver.switch_to_frame('modalIfm')
        driver.find_element_by_xpath('//*[@id="content_body"]/div/map/area').click()
        WebDriverWait(driver, 5).until(lambda d: len(driver.window_handles) == 2)
        driver.switch_to.window(driver.window_handles[1])
        driver.close()
        driver.switch_to.window(driver.window_handles[0])
    except TimeoutException:
        pass

def 파일갯수_확인():
    soup = BeautifulSoup(driver.page_source,'html.parser')
    테이블 = soup.find('td',id='dntcFileListTxt')
    본인인증인지 = 테이블.find_all('a')
    if len(본인인증인지) == 0:
        return 0
    elif 본인인증인지[0].text == '본인인증':
        return -1
    else:
        갯수 =len(본인인증인지)
        return 갯수

종결처리

In [6]:
def 종결처리_사유_확인():
    try:
        WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, '//*[@id="infoOppPrcsCn"]')))
        처리사유 = driver.find_element_by_xpath('//*[@id="infoOppPrcsCn"]').text
    except TimeoutException:
        처리사유 = '시간에러'
    except :
        처리사유 = '에러'
    return 처리사유

def 종결처리_누르기_저장(count):
    WebDriverWait(driver,5).until(EC.presence_of_element_located((By.XPATH,'//*[@id="openBillingTable"]/tbody/tr[1]/td[6]/a')))
    대학이름 = driver.find_element_by_xpath('//*[@id="openBillingTable"]/tbody/tr['+str(count)+']/td[5]').text
    처리상태 = driver.find_element_by_xpath('//*[@id="openBillingTable"]/tbody/tr['+str(count)+']/td[6]/a')
    driver.execute_script("arguments[0].click();", 처리상태)

    WebDriverWait(driver, 5).until(lambda d: len(driver.window_handles) == 2)
    driver.switch_to.window(driver.window_handles[1])

    처리사유 =종결처리_사유_확인()
    저장하기({'대학이름':대학이름,'부존재_종결사유':처리사유})

    driver.close()
    driver.switch_to.window(driver.window_handles[0])

부존재처리

In [7]:
def 부존재사유_저장():
    WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, '//*[@id="oppCn"]')))
    처리사유 = driver.find_element_by_xpath('//*[@id="oppCn"]').text
    대학이름 = driver.find_element_by_xpath('//*[@id="prcsNstNm"]').text
    저장하기({'대학이름':대학이름,'부존재_종결사유':처리사유})

In [17]:
def 부존재_순서(다운로드_여부):
    for count in range(1,11):
        try:
            청구신청_하나_누르기(count)
        except NoSuchElementException:
            break
        부존재사유_저장()
        목록버튼_누르기()
def 종결_순서(다운로드_여부):
    for count in range(1,11):
        try:
            종결처리_누르기_저장(count)
        except NoSuchElementException :
            break

def 처리완료_순서(다운로드_여부):
    for count in range(1,11):
        try:
            청구신청_하나_누르기(count)
        except NoSuchElementException:
            break
        except TimeoutException:
            break
        결정통지_누르기()
        서버통신에러()
        결정통지_저장(다운로드_여부)
        목록버튼_누르기()

def 대기중_순서(다운로드_여부):
    청구신청조회_저장()

def 작동하기(아이디, 비밀번호,원하는_처리상태,청구일자_='최근3개월',다운로드_여부='한다'):    
    접속하기(아이디, 비밀번호)
    for 상태 in 원하는_처리상태:
        조건_누르기(청구일자=청구일자_,처리상태=상태)
        순환범위 = round(int(목록갯수_확인())/10 + 0.4) 
        print(상태 +': '+목록갯수_확인())
        for 페이지 in range(0,순환범위):
            청구신청조회_저장()
            기능들[상태](다운로드_여부)
            다음페이지_누르기()
            서버통신에러()

In [ ]:
기능들 = {'접수대기중':대기중_순서,'접수완료':대기중_순서,'처리중':대기중_순서,'이송완료':종결_순서,'정보공개정지':대기중_순서,'처리완료':처리완료_순서,'통지완료':처리완료_순서,'10일초과종결':대기중_순서,'종결처리':종결_순서,'정보부존재':부존재_순서}

In [ ]:
driver = webdriver.Chrome(r'C:\chromedriver.exe',chrome_options=크롬_옵션설정_확인())
저장소 = pd.DataFrame(columns=['결정','통지(처리)날짜','공개사유','비공개사유','부존재_종결사유','데이터유무'])
작동하기('joo4458', 'ghwns328@',기능들.keys())#기능들.keys())
저장소.fillna('-').to_excel(r'C:\Users\user\Desktop\정보목록실태지도\청구신청_목록.xlsx')
driver.quit()

접수대기중: 30
접수완료: 31
처리중: 25
이송완료: 2
정보공개정지: 1
처리완료: 195


In [16]:
저장소

,결정,통지(처리)날짜,공개사유,비공개사유,부존재_종결사유,데이터유무
국민대학교,처리완료(공개),2017.09.22,본인확인을 하지 않으시면 결과(공개내용 및 공개자료) 확인이 불가 합니다.본인인증,,NaN,-1
홍익대학교,처리완료(부분공개),2017.10.19,"정보공개법 제9조제1항제5호, 제6호, 제7호에 따라 공개될 경우 업무의 공정한 수...",,NaN,1
호원대학교,처리완료(공개),2017.09.11,본인확인을 하지 않으시면 결과(공개내용 및 공개자료) 확인이 불가 합니다.본인인증,,NaN,-1
호서대학교,처리완료(공개),2017.09.29,요청하신 정보공개청구에 대한 내용을 붙임과 같이 보내드립니다. 감사합니다.,,NaN,1
호남신학대학교,처리완료(공개),2017.09.14,정보공개 요청에 따라 2017.01.01-2017.07.01(6개월) 동안 모든 부...,,NaN,1
호남대학교,처리완료(비공개),2017.09.15,,청구인은 정보 요청의 정당한 사유나 목적을 밝히지 않고 국내 모든 대학에 동일한 정...,NaN,0
혜전대학교,처리완료(공개),2017.09.29,본인확인을 하지 않으시면 결과(공개내용 및 공개자료) 확인이 불가 합니다.본인인증,,NaN,-1
한일장신대학교,처리완료(즉시공개),2017.09.19,"20171.1~2017.7.1(6개월)한일장신대학교 수,발신 문서",,NaN,2
한양여자대학교,처리완료(비공개),2017.09.22,,본 대학에서는 귀하께서 요청하신 정보공개 요청서에 대한 정보 이용 목적이 정확히 명...,NaN,0
한양대학교,처리완료(비공개),2017.09.29,,본교에서는 청구하신 정보에 대하여 근거법령: 공공기관의 정보공개에 관한 법률 시행령...,NaN,0


In [28]:
다음페이지_누르기()

0